In [26]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.utils.class_weight import compute_class_weight
import joblib
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.optimizers import Adam
from sklearn.metrics import accuracy_score

# Load dataset
df = pd.read_csv('hand_landmarks.csv')

# Assuming the first column 'label' is binary and the rest are features
X = df.drop('label', axis=1).values
y = df['label'].values

# Normalize the features
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Split the data into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42)

# Save the scaler for later use
joblib.dump(scaler, 'scaler.pkl')


['scaler.pkl']

In [27]:
# Define the model
model = Sequential([
    Dense(32, activation='relu', input_shape=(X_train.shape[1],)),
    Dropout(0.5),
    Dense(32, activation='relu'),
    Dropout(0.5),
    Dense(1, activation='sigmoid')  # For binary classification
])

model.compile(optimizer=Adam(learning_rate=0.001),
              loss='binary_crossentropy',  # For binary classification
              metrics=['accuracy'])

model.summary()


Model: "sequential_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_12 (Dense)            (None, 32)                2048      
                                                                 
 dropout_8 (Dropout)         (None, 32)                0         
                                                                 
 dense_13 (Dense)            (None, 32)                1056      
                                                                 
 dropout_9 (Dropout)         (None, 32)                0         
                                                                 
 dense_14 (Dense)            (None, 1)                 33        
                                                                 
Total params: 3137 (12.25 KB)
Trainable params: 3137 (12.25 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [28]:
# Compute class weights
class_weights = compute_class_weight('balanced', classes=np.unique(y_train), y=y_train)
class_weight_dict = {0: class_weights[0], 1: class_weights[1]}

# Train the model
model.fit(X_train, y_train, epochs=1000, validation_split=0.2, batch_size=64, class_weight=class_weight_dict)

# Save the model
model.save('hand_model.h5')


Epoch 1/1000
11/11 [==============================] - 1s 19ms/step - loss: 0.9005 - accuracy: 0.4915 - val_loss: 0.6734 - val_accuracy: 0.5926
Epoch 2/1000
11/11 [==============================] - 0s 4ms/step - loss: 0.8252 - accuracy: 0.5178 - val_loss: 0.6668 - val_accuracy: 0.5370
Epoch 3/1000
11/11 [==============================] - 0s 5ms/step - loss: 0.7650 - accuracy: 0.5255 - val_loss: 0.6625 - val_accuracy: 0.5556
Epoch 4/1000
11/11 [==============================] - 0s 4ms/step - loss: 0.7316 - accuracy: 0.5456 - val_loss: 0.6571 - val_accuracy: 0.5926
Epoch 5/1000
11/11 [==============================] - 0s 4ms/step - loss: 0.7205 - accuracy: 0.5209 - val_loss: 0.6527 - val_accuracy: 0.5988
Epoch 6/1000
11/11 [==============================] - 0s 4ms/step - loss: 0.7362 - accuracy: 0.5008 - val_loss: 0.6485 - val_accuracy: 0.6111
Epoch 7/1000
11/11 [==============================] - 0s 4ms/step - loss: 0.7136 - accuracy: 0.5440 - val_loss: 0.6449 - val_accuracy: 0.6420
Epoch

c:\Users\zachu\.conda\envs\hand_env\lib\site-packages\keras\src\engine\training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [29]:

# Predict probabilities
probabilities = model.predict(X_test)

# Define a custom threshold
threshold = 0.99  # Adjust based on your requirement

# Apply the threshold
predictions = (probabilities > threshold).astype(int)

# Evaluate accuracy
test_accuracy = accuracy_score(y_test, predictions)
print(f'Test Accuracy with threshold {threshold}:', test_accuracy)


7/7 [==============================] - 0s 1ms/step
Test Accuracy with threshold 0.99: 0.9802955665024631
